In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
canada_mobility = pd.read_csv("Canada_region_mobility_2020.csv")

In [3]:
print(canada_mobility.columns)

#Only get Ontario
province = ["Ontario"]
canada_mobility = canada_mobility[canada_mobility.sub_region_1.isin(province)]

#Only get Ottawa and Toronto cases
cities = ["Toronto Division","Ottawa Division"]
canada_mobility = canada_mobility[canada_mobility.sub_region_2.isin(cities)]

#limit the date between march and june
start_date = '2020-03-01'
end_date = '2020-06-30'
mask = (canada_mobility['date'] >= start_date) & (canada_mobility['date'] <= end_date)
canada_mobility = canada_mobility.loc[mask]

nan_value = "Ottawa"
canada_mobility.replace("Ottawa Division", nan_value, inplace=True)


nan_value = "Toronto"
canada_mobility.replace("Toronto Division", nan_value, inplace=True)

canada_mobility['place_id'] = canada_mobility['place_id'].astype(str) + "+" + canada_mobility['date'].astype(str)
canada_mobility

Index(['country_region_code', 'country_region', 'sub_region_1', 'sub_region_2',
       'metro_area', 'iso_3166_2_code', 'census_fips_code', 'place_id', 'date',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline'],
      dtype='object')


,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,place_id,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
46080,CA,Canada,Ontario,Ottawa,NaN,NaN,NaN,ChIJmcB2guBn0UwRr0RW_iic0LY+2020-03-02,2020-03-02,2.0,6.0,-7.0,-5.0,4.0,0.0
46081,CA,Canada,Ontario,Ottawa,NaN,NaN,NaN,ChIJmcB2guBn0UwRr0RW_iic0LY+2020-03-03,2020-03-03,0.0,5.0,-10.0,-2.0,4.0,0.0
46082,CA,Canada,Ontario,Ottawa,NaN,NaN,NaN,ChIJmcB2guBn0UwRr0RW_iic0LY+2020-03-04,2020-03-04,3.0,9.0,-9.0,-2.0,5.0,0.0
46083,CA,Canada,Ontario,Ottawa,NaN,NaN,NaN,ChIJmcB2guBn0UwRr0RW_iic0LY+2020-03-05,2020-03-05,9.0,8.0,-5.0,-4.0,-1.0,0.0
46084,CA,Canada,Ontario,Ottawa,NaN,NaN,NaN,ChIJmcB2guBn0UwRr0RW_iic0LY+2020-03-06,2020-03-06,-3.0,-2.0,-10.0,-1.0,5.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52815,CA,Canada,Ontario,Toronto,NaN,NaN,NaN,ChIJ5b2RG4_L1IkRDtQ2gFEjLv4+2020-06-26,2020-06-26,-38.0,-9.0,105.0,-53.0,-53.0,18.0
52816,CA,Canada,Ontario,Toronto,NaN,NaN,NaN,ChIJ5b2RG4_L1IkRDtQ2gFEjLv4+2020-06-27,2020-06-27,-38.0,-11.0,113.0,-38.0,-15.0,7.0
52817,CA,Canada,Ontario,Toronto,NaN,NaN,NaN,ChIJ5b2RG4_L1IkRDtQ2gFEjLv4+2020-06-28,2020-06-28,-36.0,-14.0,154.0,-35.0,-10.0,3.0
52818,CA,Canada,Ontario,Toronto,NaN,NaN,NaN,ChIJ5b2RG4_L1IkRDtQ2gFEjLv4+2020-06-29,2020-06-29,-33.0,-7.0,101.0,-57.0,-55.0,17.0


In [4]:
#mobility dimension
Mobility_dimension = pd.DataFrame(columns=['mobility_key','mobility_ID', 'province', 'subregion', 'metro_area','retail_and_recreation', 'grocery_and_pharmacy', 'parks', 'transit_stations', 'workplaces', 'residential'])

In [5]:
#Mobility Staging
Mobility_dimension['mobility_ID'] = canada_mobility['place_id']
Mobility_dimension['province'] = canada_mobility['sub_region_1']
Mobility_dimension['subregion'] = canada_mobility['sub_region_2']
Mobility_dimension['metro_area'] = canada_mobility['metro_area']
Mobility_dimension['retail_and_recreation'] = canada_mobility['retail_and_recreation_percent_change_from_baseline']
Mobility_dimension['grocery_and_pharmacy'] = canada_mobility['grocery_and_pharmacy_percent_change_from_baseline']
Mobility_dimension['parks'] = canada_mobility['parks_percent_change_from_baseline']
Mobility_dimension['transit_stations'] = canada_mobility['transit_stations_percent_change_from_baseline']
Mobility_dimension['workplaces'] = canada_mobility['workplaces_percent_change_from_baseline']
Mobility_dimension['residential'] = canada_mobility['residential_percent_change_from_baseline']

#Create Surrogate Keys
key = 100
for index, row in Mobility_dimension.iterrows():    
   Mobility_dimension.at[index, "mobility_key"] = key
   key += 1

#Show Dimension
Mobility_dimension.head(10)

,mobility_key,mobility_ID,province,subregion,metro_area,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential
46080,100,ChIJmcB2guBn0UwRr0RW_iic0LY+2020-03-02,Ontario,Ottawa,NaN,2.0,6.0,-7.0,-5.0,4.0,0.0
46081,101,ChIJmcB2guBn0UwRr0RW_iic0LY+2020-03-03,Ontario,Ottawa,NaN,0.0,5.0,-10.0,-2.0,4.0,0.0
46082,102,ChIJmcB2guBn0UwRr0RW_iic0LY+2020-03-04,Ontario,Ottawa,NaN,3.0,9.0,-9.0,-2.0,5.0,0.0
46083,103,ChIJmcB2guBn0UwRr0RW_iic0LY+2020-03-05,Ontario,Ottawa,NaN,9.0,8.0,-5.0,-4.0,-1.0,0.0
46084,104,ChIJmcB2guBn0UwRr0RW_iic0LY+2020-03-06,Ontario,Ottawa,NaN,-3.0,-2.0,-10.0,-1.0,5.0,1.0
46085,105,ChIJmcB2guBn0UwRr0RW_iic0LY+2020-03-07,Ontario,Ottawa,NaN,18.0,12.0,16.0,5.0,6.0,-3.0
46086,106,ChIJmcB2guBn0UwRr0RW_iic0LY+2020-03-08,Ontario,Ottawa,NaN,16.0,20.0,30.0,0.0,6.0,-2.0
46087,107,ChIJmcB2guBn0UwRr0RW_iic0LY+2020-03-09,Ontario,Ottawa,NaN,4.0,8.0,11.0,-8.0,3.0,0.0
46088,108,ChIJmcB2guBn0UwRr0RW_iic0LY+2020-03-10,Ontario,Ottawa,NaN,-2.0,4.0,-9.0,-8.0,3.0,1.0
46089,109,ChIJmcB2guBn0UwRr0RW_iic0LY+2020-03-11,Ontario,Ottawa,NaN,2.0,12.0,-7.0,-5.0,3.0,0.0


In [6]:
#Create csv file for Patient dimension
Mobility_dimension.to_csv("Mobility_dimension.csv",index=False)